In [70]:
from voice_agent_flow.llms import create_pydantic_azure_openai
import nest_asyncio
from pydantic_ai import Agent, AgentRunResultEvent, AgentStreamEvent
nest_asyncio.apply()
from pprint import pprint
import asyncio

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(model)




**Synchronous Run**

In [71]:
result_sync = agent.run_sync('What is the capital of Italy?')
print(result_sync.output)

The capital of Italy is **Rome**.


**Asynchronous Run**

In [72]:
await agent.run('What is the capital of France?')

AgentRunResult(output='The capital of France is **Paris**.')

**Asyncronous Streaming Text**

In [73]:
async with agent.run_stream('What is the capital of the UK?') as response:
        async for text in response.stream_text(delta = True):
            print(text)

The capital of the UK is **London**.


**Asynchronous Streaming Events**

In [77]:
from pydantic import BaseModel

class Person(BaseModel):
    """Defines a person"""
    name:str
    aget:str
    
events: list[AgentStreamEvent | AgentRunResultEvent] = []

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(
    model,
    output_type= str | Person,
    instructions = "You are a helpful assistant that answers questions about anything, but when the user metions a person, you will reply with a json object defined by the schema."

)


async for event in agent.run_stream_events('My name is huan, 32 years old.'):
    events.append(event)
    print(event)

PartStartEvent(index=0, part=ToolCallPart(tool_name='final_result', args='', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj'))
FinalResultEvent(tool_name='final_result', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj')
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='{"', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='name', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='":"', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='hu', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='an', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='","', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='aget', tool_call_id='call_sA26CBB7wuDFrqzK1n3Q7VEj'

In [6]:
def weather_tool(location: str) -> str:
    """Search the weather for a location."""
    return f"The weather in {location} is sunny."

agent = Agent(model, tools = [weather_tool])

events = []
async for event in agent.run_stream_events('What is the weather in New York City?'):
    print(event)
    events.append(event)

PartStartEvent(index=0, part=ToolCallPart(tool_name='weather_tool', args='', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='{"', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='location', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='":"', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='New', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta=' York', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta=' City', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='"}', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD'))
PartEndEvent(index=0, part=ToolCallPart(tool_name='weather_tool', args='{"loca

In [7]:
from pydantic_ai import ToolCallPart, PartStartEvent

In [8]:
from pydantic_ai import (
    AgentStreamEvent,
    FinalResultEvent,
    FunctionToolCallEvent,
    FunctionToolResultEvent,
    PartDeltaEvent,
    PartStartEvent,
    RunContext,
    TextPartDelta,
    ThinkingPartDelta,
    ToolCallPartDelta,
    TextPart,
    PartEndEvent,
    AgentRunResult,
    AgentRunResultEvent
)


# async def handle_event(event: AgentStreamEvent):
#     if isinstance(event, PartStartEvent):
#         output_messages.append(f'[Request] Starting part {event.index}: {event.part!r}')
#     elif isinstance(event, PartDeltaEvent):
#         if isinstance(event.delta, TextPartDelta):
#             output_messages.append(f'[Request] Part {event.index} text delta: {event.delta.content_delta!r}')
#         elif isinstance(event.delta, ThinkingPartDelta):
#             output_messages.append(f'[Request] Part {event.index} thinking delta: {event.delta.content_delta!r}')
#         elif isinstance(event.delta, ToolCallPartDelta):
#             output_messages.append(f'[Request] Part {event.index} args delta: {event.delta.args_delta}')
#     elif isinstance(event, FunctionToolCallEvent):
#         output_messages.append(
#             f'[Tools] The LLM calls tool={event.part.tool_name!r} with args={event.part.args} (tool_call_id={event.part.tool_call_id!r})'
#         )
#     elif isinstance(event, FunctionToolResultEvent):
#         output_messages.append(f'[Tools] Tool call {event.tool_call_id!r} returned => {event.result.content}')
#     elif isinstance(event, FinalResultEvent):
#         output_messages.append(f'[Result] The model starting producing a final result (tool_name={event.tool_name})')


In [9]:
from asyncio import Queue

# 判断是否为part开始
is_part_start = lambda event: isinstance(event, PartStartEvent)
# 如果是part开始，判断part是tool还是text(必须先判断是不是part开始)
is_tool_arg_start =  lambda event: isinstance(event.part, ToolCallPart)
is_text_start = lambda event: isinstance(event.part, TextPart) 

# 判断是否为part结束
is_part_end = lambda event: isinstance(event, PartEndEvent)
# 判断part结束是tool还是text(必须先判断是不是part结束)
is_tool_arg_end = lambda event: isinstance(event.delta, ToolCallPart)
is_text_end = lambda event: isinstance(event.delta, TextPart)

# 判断是不是delta
is_delta = lambda event: isinstance(event, PartDeltaEvent)
# 判断delta是tool还是text(必须先判断是不是delta)
is_tool_arg_delta = lambda event: isinstance(event.delta, ToolCallPartDelta)
is_text_delta = lambda event: isinstance(event.delta, TextPartDelta)

# 判断是否为tool调用事件
is_function_tool_call_args = lambda event: isinstance(event, FunctionToolCallEvent)
# 判断是否为tool结果事件
is_function_tool_result = lambda event: isinstance(event, FunctionToolResultEvent)

# 判断是否为最终结果事件
is_final_result = lambda event: isinstance(event, FinalResultEvent)

is_agent_run_result = lambda event: isinstance(event, AgentRunResultEvent)

queue = Queue()

async def handle_event(event, queue:Queue):
    
    if is_part_start(event) and is_tool_arg_start(event):
        await queue.put(f'[Request] Starting tool call part {event.index}: {event.part!r}')
    
    elif is_part_start(event) and is_text_start(event):
        await queue.put(f'[Request] Starting text part {event.index}: {event.part!r}')
    
    elif is_delta(event) and is_tool_arg_delta(event):
        await queue.put(f'[Request] Part {event.index} tool args delta: {event.delta.args_delta}')
     
    elif is_delta(event) and is_text_delta(event):
        await queue.put(f'[Request] Part {event.index} text delta: {event.delta.content_delta!r}')
    
    elif is_function_tool_call_args(event):
        await queue.put(
            f'[Tools] The LLM calls tool={event.part.tool_name!r} with args={event.part.args} (tool_call_id={event.part.tool_call_id!r})'
        )
        
    elif is_part_end(event):
        if is_tool_arg_end(event):
            print("tool call arg output done.")
        elif is_text_end(event):
            print()
            print("text output done.")
        
    elif is_function_tool_result(event):
        await queue.put(f'[Tools] Tool call {event.tool_call_id!r} returned => {event.result.content}')
    
    elif is_final_result(event):
        await queue.put(f'[Result] The model starting producing a final result (tool_name={event.tool_name})')
        
    else:
        print(f'Unhandled event: {event}')
        

In [10]:
import asyncio

# 判断是否为part开始
is_part_start = lambda event: isinstance(event, PartStartEvent)
# 如果是part开始，判断part是tool还是text(必须先判断是不是part开始)
is_tool_arg_start =  lambda event: isinstance(event.part, ToolCallPart)
is_text_start = lambda event: isinstance(event.part, TextPart) or isinstance(event.part, TextPartDelta)

# 判断是否为part结束
is_part_end = lambda event: isinstance(event, PartEndEvent)
# 判断part结束是tool还是text(必须先判断是不是part结束)
is_tool_arg_end = lambda event: isinstance(event.part, ToolCallPart)
is_text_end = lambda event: isinstance(event.part, TextPart)

# 判断是不是delta
is_delta = lambda event: isinstance(event, PartDeltaEvent)
# 判断delta是tool还是text(必须先判断是不是delta)
is_tool_arg_delta = lambda event: isinstance(event.delta, ToolCallPartDelta)
is_text_delta = lambda event: isinstance(event.delta, TextPartDelta)

# 判断是否为tool调用事件
is_function_tool_call_args = lambda event: isinstance(event, FunctionToolCallEvent)
# 判断是否为tool结果事件
is_function_tool_result = lambda event: isinstance(event, FunctionToolResultEvent)

# 判断是否为最终结果事件
is_final_result = lambda event: isinstance(event, FinalResultEvent)

# 判断是否为agent结果输出
is_agent_run_result = lambda event: isinstance(event, AgentRunResultEvent)

class AgentRunner:
    
    def __init__(self, agent, queue:asyncio.Queue = None):
        
        self.agent:Agent = agent
        self.queue = queue if queue else asyncio.Queue()
        self.final_result = False
        
    async def run_sync(self, prompt: str):
        self.final_result = False
        async for event in self.agent.run_stream_events(prompt):
            await self.handle_event(event)
        await self.print_events()
    
    async def handle_event(self, event):
        
        if is_part_start(event) and is_tool_arg_start(event):
            await self.queue.put(event)
            print("🤔 等下，我想想")
        
        elif is_part_start(event) and is_text_start(event):
            await self.queue.put(event)
            print(event.part.content, end = "😄")
                
        elif is_delta(event) and is_tool_arg_delta(event):
            await self.queue.put(event)
        
        elif is_delta(event) and is_text_delta(event):
            await self.queue.put(event)
            if self.final_result:
                print(event.delta.content_delta, end = "😄")
                
        elif is_part_end(event):
            if is_tool_arg_end(event):
                print("tool call arg output done.")
            elif is_text_end(event):
                print()
                print("text output done.")
        
        elif is_function_tool_call_args(event):
            await self.queue.put(event)
            
        elif is_function_tool_result(event):
            await self.queue.put(event)
        
        elif is_final_result(event):
            await self.queue.put(event) 
            self.final_result = True
        else:
            print(f'Unhandled event: {event}')
            
    async def print_events(self):
        while not self.queue.empty():
            event =  await self.queue.get()
            print(event)
            self.queue.task_done()
            
            
runner = AgentRunner(agent)
await runner.run_sync('What is the weather in New York City?')

🤔 等下，我想想
tool call arg output done.
The😄 weather😄 in😄 New😄 York😄 City😄 is😄 currently😄 sunny😄.😄 If😄 you😄 need😄 more😄 details😄,😄 like😄 temperature😄 or😄 forecast😄,😄 let😄 me😄 know😄!😄
text output done.
Unhandled event: AgentRunResultEvent(result=AgentRunResult(output='The weather in New York City is currently sunny. If you need more details, like temperature or forecast, let me know!'))
PartStartEvent(index=0, part=ToolCallPart(tool_name='weather_tool', args='', tool_call_id='call_jXDjPOyGg6tp5CoztJJBqCNl'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='{"', tool_call_id='call_jXDjPOyGg6tp5CoztJJBqCNl'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='location', tool_call_id='call_jXDjPOyGg6tp5CoztJJBqCNl'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='":"', tool_call_id='call_jXDjPOyGg6tp5CoztJJBqCNl'))
PartDeltaEvent(index=0, delta=ToolCallPartDelta(args_delta='New', tool_call_id='call_jXDjPOyGg6tp5CoztJJBqCNl'))
PartDeltaEvent(index=0, delta=ToolC

In [24]:
agent = Agent(model, tools = [weather_tool])

result = agent.run_sync('What is the weather in New York City?')

In [25]:
result.all_messages()

[ModelRequest(parts=[UserPromptPart(content='What is the weather in New York City?', timestamp=datetime.datetime(2026, 2, 18, 15, 2, 25, 216128, tzinfo=datetime.timezone.utc))], run_id='02eb26dd-9a1e-40c9-9a80-8ab21c2fa3c9'),
 ModelResponse(parts=[ToolCallPart(tool_name='weather_tool', args='{"location":"New York City"}', tool_call_id='call_W4o1rrmbJZkHnNjyDY6tDzmH')], usage=RequestUsage(input_tokens=53, output_tokens=17, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 18, 15, 2, 26, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_details={'finish_reason': 'tool_calls'}, provider_response_id='chatcmpl-DAdMI1kl1My23Vf2h60zWzL58UiUG', finish_reason='tool_call', run_id='02eb26dd-9a1e-40c9-9a80-8ab21c2fa3c9'),
 ModelRequest(parts=[ToolReturnPart(tool_name='weather

In [ ]:


from pydantic_core import to_jsonable_python

from pydantic_ai import (
    Agent,
    ModelMessagesTypeAdapter,  
)


In [13]:
jsonable = to_jsonable_python(result.all_messages())

In [14]:
msg_json = to_jsonable_python(result.all_messages()[3])

In [15]:
msg_json = to_jsonable_python(result.all_messages()[3:4])
ModelMessagesTypeAdapter.validate_python(msg_json)


[ModelResponse(parts=[TextPart(content='The weather in New York City is currently sunny. If you need more details like temperature or forecast, let me know!')], usage=RequestUsage(input_tokens=86, output_tokens=26, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 18, 14, 59, 17, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-DAdJFKH8RpNDJjiqP8H0Tunr26eoO', finish_reason='stop', run_id='19af42c7-51cb-46a4-aced-c2ab2a43b527')]

In [16]:
to_jsonable_python(result)

{'output': 'The weather in New York City is currently sunny. If you need more details like temperature or forecast, let me know!',
 '_output_tool_name': None,
 '_state': {'message_history': [{'parts': [{'content': 'What is the weather in New York City?',
      'timestamp': '2026-02-18T14:59:15.567383Z',
      'part_kind': 'user-prompt'}],
    'instructions': None,
    'kind': 'request',
    'run_id': '19af42c7-51cb-46a4-aced-c2ab2a43b527',
    'metadata': None},
   {'parts': [{'tool_name': 'weather_tool',
      'args': '{"location":"New York City"}',
      'tool_call_id': 'call_Esdj4V3h1ue5rka0dJWTkDIP',
      'id': None,
      'provider_details': None,
      'part_kind': 'tool-call'}],
    'usage': {'input_tokens': 53,
     'cache_write_tokens': 0,
     'cache_read_tokens': 0,
     'output_tokens': 17,
     'input_audio_tokens': 0,
     'cache_audio_read_tokens': 0,
     'output_audio_tokens': 0,
     'details': {'accepted_prediction_tokens': 0,
      'audio_tokens': 0,
      'reasoni

In [ ]:
ModelMessagesTypeAdapter.validate_python([])

ValidationError: 1 validation error for list[tagged-union[ModelRequest,ModelResponse]]
0
  Unable to extract tag using discriminator 'kind' [type=union_tag_not_found, input_value={'tool_name': 'weather_to...part_kind': 'tool-call'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/union_tag_not_found

In [45]:
from pydantic_ai import ModelRequest, ToolCallPart, ModelResponse

ModelRequest(parts = [events[9].part])

ModelRequest(parts=[ToolCallPart(tool_name='weather_tool', args='{"location":"New York City"}', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD')])

In [37]:
ModelRequest(ToolCallPart(tool_name = "weather_tool", args = '{"location": "New York City"}', tool_call_id = "1234"))

ModelRequest(parts=ToolCallPart(tool_name='weather_tool', args='{"location": "New York City"}', tool_call_id='1234'))

In [41]:
events[10].result

ToolReturnPart(tool_name='weather_tool', content='The weather in New York City is sunny.', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD', timestamp=datetime.datetime(2026, 2, 18, 14, 59, 9, 619076, tzinfo=datetime.timezone.utc))

In [43]:
to_jsonable_python(events[9].part)

{'tool_name': 'weather_tool',
 'args': '{"location":"New York City"}',
 'tool_call_id': 'call_0tEPo1T8sE1GvoibSww0pqlD',
 'id': None,
 'provider_details': None,
 'part_kind': 'tool-call'}

In [44]:
to_jsonable_python(events[10].result)

{'tool_name': 'weather_tool',
 'content': 'The weather in New York City is sunny.',
 'tool_call_id': 'call_0tEPo1T8sE1GvoibSww0pqlD',
 'metadata': None,
 'timestamp': '2026-02-18T14:59:09.619076Z',
 'part_kind': 'tool-return'}

In [48]:
to_jsonable_python(events[9].part)

{'tool_name': 'weather_tool',
 'args': '{"location":"New York City"}',
 'tool_call_id': 'call_0tEPo1T8sE1GvoibSww0pqlD',
 'id': None,
 'provider_details': None,
 'part_kind': 'tool-call'}

In [52]:
to_jsonable_python(events[10].result)

{'tool_name': 'weather_tool',
 'content': 'The weather in New York City is sunny.',
 'tool_call_id': 'call_0tEPo1T8sE1GvoibSww0pqlD',
 'metadata': None,
 'timestamp': '2026-02-18T14:59:09.619076Z',
 'part_kind': 'tool-return'}

In [ ]:
from pydantic_ai import ToolReturnPart
from pydantic_ai import ModelRequest
tool_return_part = ToolReturnPart(
    **{
        'tool_name': 'weather_tool',
        'content': 'The weather in New York City is sunny.',
        'tool_call_id': 'call_0tEPo1T8sE1GvoibSww0pqlD',
        'metadata': None,
        'timestamp': '2026-02-18T14:59:09.619076Z',
        'part_kind': 'tool-return'
    }
)

model_request_part = ModelRequest(parts = [tool_return_part])
model_request_part


ModelRequest(parts=[ToolReturnPart(tool_name='weather_tool', content='The weather in New York City is sunny.', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD', timestamp='2026-02-18T14:59:09.619076Z')])

In [67]:
tool_call_part = ToolCallPart(
    **{
        'tool_name': 'weather_tool',
        'args': '{"location":"New York City"}',
        'tool_call_id': 'call_0tEPo1T8sE1GvoibSww0pqlD',
        'id': None,
        'provider_details': None,
        'part_kind': 'tool-call'
    }
)


model_response_part = ModelResponse(parts = [tool_call_part])
ModelMessagesTypeAdapter.validate_python([to_jsonable_python(model_response_part)])

[ModelResponse(parts=[ToolCallPart(tool_name='weather_tool', args='{"location":"New York City"}', tool_call_id='call_0tEPo1T8sE1GvoibSww0pqlD')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 18, 15, 21, 3, 233501, tzinfo=TzInfo(UTC)))]